## pipeline

In [1]:
import numpy as np
import os
import pickle
import copy
import edt
import matplotlib.pyplot as plt
import time
import cv2
import pandas as pd
from sklearn.metrics.cluster import adjusted_rand_score
from skimage.metrics import adapted_rand_error

import torch
from torch import from_numpy as from_numpy
from torchsummary import summary

from func.run_pipeline_super_vox import segment_super_vox_3_channel, semantic_segment_crop_and_cat_3_channel_output, \
img_3d_erosion_or_expansion, segment_super_vox_2_channel, semantic_segment_crop_and_cat_2_channel_output, \
generate_super_vox_by_watershed, get_outlayer_of_a_3d_shape, get_crop_by_pixel_val, Cluster_Super_Vox, assign_boudary_voxels_to_cells_with_watershed, \
delete_too_small_cluster, reassign
from func.run_pipeline import segment, assign_boudary_voxels_to_cells, dbscan_of_seg, semantic_segment_crop_and_cat
from func.cal_accuracy import IOU_and_Dice_Accuracy, VOI
from func.network import VoxResNet, CellSegNet_basic_lite
from func.unet_3d_basic import UNet3D_basic
from func.ultis import save_obj, load_obj

### init model

In [2]:
# model=UNet3D_basic(in_channels = 1, out_channels = 3)
# load_path=''
# model=VoxResNet(input_channel=1, n_classes=3, output_func = "softmax")
# load_path=''
model=CellSegNet_basic_lite(input_channel=1, n_classes=3, output_func = "softmax")
load_path='output/model_Ovules.pkl'
checkpoint = torch.load(load_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

CellSegNet_basic_lite(
  (conv1): Conv3d(1, 16, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (conv2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bnorm1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (resmodule1): ResModule(
    (batchnorm_module): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_module): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  )
  (conv4): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (resmodule2): ResModule(
    (batchnorm_module): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_module): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  )
  (conv5): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (resmodule3): R

### dataset info

In [3]:
Ovules_data_dict = load_obj("dataset_info/Ovules_dataset_info")

In [4]:
Ovules_data_dict

{'train': {'N_509_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_509_ds2x.npz',
  'N_449_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_449_ds2x.npz',
  'N_536_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_536_ds2x.npz',
  'N_454_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_454_ds2x.npz',
  'N_422_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_422_ds2x.npz',
  'N_416_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_416_ds2x.npz',
  'N_428_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_428_ds2x.npz',
  'N_512_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_512_ds2x.npz',
  'N_425_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_425_ds2x.npz',
  'N_405_B_ds2x.npz': 'data/CellSeg_dataset/Ovules_processed_thin_boundary/train/N_405_B_ds2x.npz',
  'N_458_ds2x.npz': 'data/C

### seg one img

parameter setting

In [5]:
# we do not input the whole raw image to the model one time but input raw image crops
crop_cube_size=128
stride=64

# hyperparameter for TASCAN, min touching area of two super pixels if they belong to the same cell
# min_touching_area=30

choose a test image and load it

In [6]:
print("there are test imgs: "+str(Ovules_data_dict['test'].keys()))
case = 'N_435_final_crop_ds2.npz'
print("for test case "+str(case)+" : "+str(Ovules_data_dict['test'][case]))

there are test imgs: dict_keys(['N_441_final_crop_ds2.npz', 'N_522_final_crop_ds2.npz', 'N_511_final_crop_ds2.npz', 'N_435_final_crop_ds2.npz', 'N_294_final_crop_ds2.npz', 'N_590_final_crop_ds2.npz', 'N_593_final_crop_ds2.npz'])
for test case N_435_final_crop_ds2.npz : data/CellSeg_dataset/Ovules_processed_thin_boundary/test/N_435_final_crop_ds2.npz


In [7]:
hf = np.load(Ovules_data_dict['test'][case])
raw_img = np.array(hf["raw"], dtype=np.float)
hand_seg = np.array(hf["ins"], dtype=np.float)
boundary_gt = np.array(hf["boundary"], dtype=np.float)
foreground_gt = np.array(hf["foreground"], dtype=np.float)

print("raw_img shape: "+str(raw_img.shape))
print("hand_seg shape: "+str(hand_seg.shape))
raw_img_shape = raw_img.shape
hand_seg_shape = hand_seg.shape

/var/folders/qq/9q6wphrj19z8xt4qcffm8df80000gn/T/ipykernel_69640/2859134447.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_img = np.array(hf["raw"], dtype=np.float)
/var/folders/qq/9q6wphrj19z8xt4qcffm8df80000gn/T/ipykernel_69640/2859134447.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  hand_seg = np.array(hf["ins"], dtype=n

raw_img shape: (194, 550, 555)
hand_seg shape: (194, 550, 555)


load model outputs

In [8]:
# feed the raw img to the model
print('Feed raw img to model. Use different transposes')
raw_img_size=raw_img.shape
    
seg_foreground_comp = np.zeros(raw_img_size)
seg_boundary_comp = np.zeros(raw_img_size)

transposes = [[0,1,2]]#,[2,0,1],[0,2,1]]
reverse_transposes = [[0,1,2]]#,[1,2,0],[0,2,1]]

with open('seg_boundary_comp.pickle', 'rb') as f:
    seg_boundary_comp = pickle.load(f)
with open('seg_foreground_comp.pickle', 'rb') as f:
    seg_foreground_comp = pickle.load(f)
with open('seg_background_comp.pickle', 'rb') as f:
    seg_background_comp = pickle.load(f)

Feed raw img to model. Use different transposes


TASCAN

generate super vox by watershed

In [9]:
# Generate super vox by watershed
how_close_are_the_super_vox_to_boundary=2
min_touching_percentage=0.51
#min_touching_percentage=0.02

# seg_foreground_erosion=1-img_3d_erosion_or_expansion(1-seg_foreground_comp, kernel_size=how_close_are_the_super_vox_to_boundary+1, device=device)
# seg_foreground_super_voxel_by_ws = generate_super_vox_by_watershed(seg_foreground_erosion)


super voxel clustering

In [10]:
import multiprocess

In [11]:
def img_3d_interpolate(img_3d, output_size, device = torch.device('cpu'), mode='nearest'):
    img_3d = img_3d.reshape(1,1,img_3d.shape[0],img_3d.shape[1],img_3d.shape[2])
    img_3d=torch.from_numpy(img_3d).float().to(device)
    img_3d=torch.nn.functional.interpolate(img_3d, size=output_size, mode='nearest')
    img_3d=img_3d.detach().cpu().numpy()
    img_3d=img_3d.reshape(img_3d.shape[2],img_3d.shape[3],img_3d.shape[4])

    return img_3d

In [12]:
result_df_final = pd.DataFrame(columns=["min_touching_area", "split", "merge"])
result_list = []

In [ ]:
for value in range(10):
    seg_foreground_erosion=1-img_3d_erosion_or_expansion(1-seg_foreground_comp, kernel_size=how_close_are_the_super_vox_to_boundary+1, device=device)
    seg_foreground_super_voxel_by_ws = generate_super_vox_by_watershed(seg_foreground_erosion)

    min_touching_area = (value+1)*5
    # Super voxel clustering
    cluster_super_vox=Cluster_Super_Vox(min_touching_area=min_touching_area, min_touching_percentage=min_touching_percentage)
    cluster_super_vox.fit(seg_foreground_super_voxel_by_ws)
    seg_foreground_single_cell_with_boundary = cluster_super_vox.output_3d_img

    # Delete too small cells
    min_cell_size_threshold=10

    seg_foreground_single_cell_with_boundary_delete_too_small = delete_too_small_cluster(seg_foreground_single_cell_with_boundary, threshold=min_cell_size_threshold)

    # Assign boudary voxels to their nearest cells
    seg_final=assign_boudary_voxels_to_cells_with_watershed(seg_foreground_single_cell_with_boundary_delete_too_small, seg_boundary_comp, seg_background_comp, compactness=1)

    # Reassign unique numbers
    seg_final=reassign(seg_final)

    # Evaluate with Metrics

    import func.cal_accuracy

    #accuracy_result = iou_and_dice_accuracy.cal_accuracy()
    voi = VOI(seg_final.astype(int), hand_seg.astype(int))

    print(f"VOI for {min_touching_area}:")
    print(voi)

    accuracy=IOU_and_Dice_Accuracy(img_3d_interpolate(hand_seg, output_size = seg_final.shape), img_3d_interpolate(seg_final, output_size = seg_final.shape))
    accuracy_record=accuracy.cal_accuracy_II()
    iou=np.array(accuracy_record[:,1]>0.7, dtype=float)
    print(str(sum(iou)/len(iou)))
    dice=np.array(accuracy_record[:,2]>0.7, dtype=float)
    print(str(sum(dice)/len(dice)))

    #result_df = result_df_final.append({"min_touching_area": min_touching_area,
    #                              "split": voi[0],
    #                              "merge": voi[1]},
    #                             ignore_index=True)
    result_list.append({"min_touching_area": min_touching_area,
                        "split": voi[0],
                        "merge": voi[1],
                        "iou": str(sum(iou)/len(iou)),
                        "dice": str(sum(dice)/len(dice))})



#pool_obj = multiprocess.Pool(8)
#pool_obj.map(run_voi, range(18))

_area: 0current_crop_outlayer_area: 843current_crop_outlayer_area: 78current_crop_outlayer_area: 480current_crop_outlayer_area: 908current_crop_outlayer_area: 0current_crop_outlayer_area: 0current_crop_outlayer_area: 67current_crop_outlayer_area: 275current_crop_outlayer_area: 275current_crop_outlayer_area: 191current_crop_outlayer_area: 0current_crop_outlayer_area: 0current_crop_outlayer_area: 365current_crop_outlayer_area: 0current_crop_outlayer_area: 0current_crop_outlayer_area: 0current_crop_outlayer_area: 2current_crop_outlayer_area: 0current_crop_outlayer_area: 349current_crop_outlayer_area: 393current_crop_outlayer_area: 380current_crop_outlayer_area: 422current_crop_outlayer_area: 0current_crop_outlayer_area: 4current_crop_outlayer_area: 750current_crop_outlayer_area: 4current_crop_outlayer_area: 398current_crop_outlayer_area: 382current_crop_outlayer_area: 637current_crop_outlayer_area: 278current_crop_outlayer_area: 363current_crop_outlayer_area: 422current_crop_outlayer_area

In [ ]:
result_list